In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [54]:
df_train = pd.read_csv(r"data/train.csv")
df_test = pd.read_csv(r"data/test.csv")

In [55]:
df_train["Age"].fillna(round(df_train["Age"].median()), inplace=True)
df_test["Age"].fillna(round(df_test["Age"].median()), inplace=True)

df_train['Sex'] = df_train['Sex'].map({'male': 0, 'female': 1})
df_test['Sex'] = df_train['Sex'].map({'male': 0, 'female': 1})

In [56]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
X = df_train[features] # Features (entrada do modelo)
y = df_train['Survived'] # Target (o que queremos prever)

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print(f'Acuracia: {accuracy_score(y_val, y_pred)}')

Acuracia: 0.8044692737430168


In [70]:
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
X_test = df_test[features]
test_predictions = model.predict(X_test)

In [71]:
submission = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': test_predictions})
submission.to_csv('submission.csv', index=False)

In [68]:
display(y_val)


709    1
439    0
840    0
720    1
39     1
      ..
433    0
773    0
25     1
84     1
10     1
Name: Survived, Length: 179, dtype: int64